# DDL for Gold Layer

This script creates views for the Gold layer in the data warehouse. 
The Gold layer represents the final dimension and fact tables (Star Schema)

Each view performs transformations and combines data from the Silver layer 
to produce a clean, enriched, and business-ready dataset.


### Create Dimension: sales_project.gold.dim_customers

In [0]:
%sql
CREATE OR REPLACE VIEW 
    sales_project.gold.dim_customers AS
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.cst_id) AS customer_key,
    ci.cst_id AS customer_id,
    ci.cst_key AS customer_number,
    ci.cst_firstname AS first_name,
    ci.cst_lastname AS last_name,
    la.cntry AS country,
    ci.cst_marital_status AS marital_status,
    CASE
        WHEN ci.cst_gndr != "n/a" THEN ci.cst_gndr
        ELSE COALESCE(ca.gen, "n/a")
    END AS gender,    
    ca.bdate AS birthdate,
    ci.cst_create_date AS create_date
FROM
    sales_project.silver.crm_cust_info ci
LEFT JOIN sales_project.silver.erp_cust_az_12 ca
    ON ci.cst_key = ca.cid
LEFT JOIN sales_project.silver.erp_loc_a_101 la
    ON ci.cst_key = la.cid;

### Create Dimension: sales_project.gold.dim_products

In [0]:
%sql
CREATE OR REPLACE VIEW 
    sales_project.gold.dim_products AS
SELECT
    ROW_NUMBER() OVER (ORDER BY prd_start_dt, prd_key) AS product_key,
    pi.prd_id AS product_id,
    pi.prd_key AS product_number,
    pi.prd_nm AS product_name,
    pi.cat_id AS category_id,
    pc.cat AS category,
    pc.subcat AS subcategory,
    pc.maintenance,
    pi.prd_cost AS cost,
    pi.prd_line AS product_line,
    pi.prd_start_dt AS start_date       
FROM 
    sales_project.silver.crm_prd_info pi
LEFT JOIN 
    sales_project.silver.erp_px_cat_g_1_v_2 pc
    ON pi.cat_id = pc.id
WHERE 
    pi.prd_end_dt IS NULL; -- Filter out all historical data

### Create Fact: sales_project.gold.fact_sales

In [0]:
%sql
CREATE OR REPLACE VIEW 
    sales_project.gold.fact_sales AS
SELECT
    sd.sls_ord_num AS order_number,
    pr.product_key,
    cu.customer_key,
    sd.sls_order_dt AS order_date,
    sd.sls_ship_dt AS shipping_date,
    sd.sls_due_dt AS due_date,
    sd.sls_sales AS sales,
    sd.sls_quantity AS quantity,
    sd.sls_price AS price
FROM
    sales_project.silver.crm_sales_details sd
LEFT JOIN sales_project.gold.dim_products pr
    ON sd.sls_prd_key = pr.product_number  
LEFT JOIN sales_project.gold.dim_customers cu
    ON sd.sls_cust_id = cu.customer_id
ORDER BY sd.sls_ord_num;